<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module2-sampling-confidence-intervals-and-hypothesis-testing/LS_DS_142_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [0]:
import numpy as np
from scipy.stats import chisquare  # One-way chi square test

# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation

ind_obs = np.array([[1, 1], [2, 2]]).T
print(ind_obs)
print(chisquare(ind_obs, axis=None))

dep_obs = np.array([[16, 18, 16, 14, 12, 12], [32, 24, 16, 28, 20, 24]]).T
print(dep_obs)
print(chisquare(dep_obs, axis=None))

[[1 2]
 [1 2]]
Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)
[[16 32]
 [18 24]
 [16 16]
 [14 28]
 [12 20]
 [12 24]]
Power_divergenceResult(statistic=23.31034482758621, pvalue=0.015975692534127565)


In [0]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=38.69323106073592, pvalue=3.961609200867749e-09)


In [0]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Live Lecture - let's explore some more of scipy.stats

In [0]:
# Taking requests! Come to lecture with a topic or problem and we'll try it.

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.

In [7]:
# from http://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29 
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip drugsCom_raw.zip

--2018-12-04 19:21:35--  http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/zip]
Saving to: ‘drugsCom_raw.zip.1’

drugsCom_raw.zip.1  100%[===================>]  41.00M  4.54MB/s    in 11s     

2018-12-04 19:21:46 (3.66 MB/s) - ‘drugsCom_raw.zip.1’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [10]:
!ls
test_url = "drugsComTest_raw.tsv"
train_url = "drugsComTrain_raw.tsv"

'Drug+Review+Dataset+(Drugs.com)'   drugsComTest_raw.tsv
 drugsCom_raw.zip		    drugsComTrain_raw.tsv
 drugsCom_raw.zip.1		    sample_data


In [67]:
import pandas as pd

df = pd.read_table(train_url)
df.head()

# df.condition.value_counts()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [100]:
df[df.condition=='ADHD']

adhd = df[df.condition=='ADHD']

# adhd.groupby('drugName').head()

top5 = adhd.drugName.value_counts()[:5].index
bottom5 = adhd.drugName.value_counts()[-5:].index

top5_df = adhd[adhd.drugName.isin(top5)]
bot5_df = adhd[adhd.drugName.isin(bottom5)]

## proof that every value is indeed 'ADHD'
assert top5_df.condition.value_counts()[0] == top5_df.shape[0]
assert bot5_df.condition.value_counts()[0] == bot5_df.shape[0]

adhd.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
27,78563,Daytrana,ADHD,"""Hi all, My son who is 12 was diagnosed when h...",10.0,"January 12, 2017",11
51,135645,Intuniv,ADHD,"""Intuniv did not work for my son; he was bounc...",1.0,"July 21, 2011",23
116,61002,Focalin,ADHD,"""My 9 year old son has been on various medicin...",9.0,"January 31, 2010",48
211,37533,Vyvanse,ADHD,"""Today was my first day taking Vyvanse and wow...",9.0,"August 31, 2011",11


# I'm gonna make my hypothesis have something to do with whether the most popular ones are highest rated. 

In [105]:
import seaborn as sns
import altair as alt
# sns.pairplot(adhd, hue='drugName'); 

# i only care about drugName and rating. let's get rid of useless features. 
# i'll go ahead and overwrite them. 

feats_of_interest = ['drugName', 'rating', 'usefulCount']

top5_df = top5_df[feats_of_interest]
bot5_df = bot5_df[feats_of_interest]

top5_df.shape, bot5_df.shape # oh crap

((1743, 3), (5, 3))

In [138]:

highlight = alt.selection(type='single', on='mouseover',
                          fields=['symbol'], nearest=True)

# base = alt.Chart(adhd[feats_of_interest]).encode(
#   x = alt.X('drugName'),
#   y = 'rating',
#   color = 'usefulCount',
#   size = 'count()'
# )

# points = base.mark_circle().encode(
#     opacity=alt.value(0)
# ).add_selection(
#     highlight
# ).properties(
#     width=600
# )

# lines = base.mark_point().encode(
#     size=alt.condition(~highlight, alt.value(1), alt.value(30))
# )






C = alt.Chart(adhd[feats_of_interest]).mark_point().encode(
  x = 'rating',
  y = 'drugName',
  size = 'usefulCount',
  # size = 'count()', 
  tooltip=['drugName', 'rating', 'count()']
).interactive()

# points + lines
C

Chart({
  data:                                drugName  rating  usefulCount
  1                            Guanfacine     8.0          192
  27                             Daytrana    10.0           11
  51                              Intuniv     1.0           23
  116                             Focalin     9.0           48
  211                             Vyvanse     9.0           11
  250                    Lisdexamfetamine    10.0           65
  268     Amphetamine / dextroamphetamine    10.0           15
  313                         Atomoxetine     9.0           23
  376     Amphetamine / dextroamphetamine    10.0            1
  382                            Concerta    10.0           43
  384     Amphetamine / dextroamphetamine     9.0           10
  385                           Strattera     3.0            8
  501                          Focalin XR     8.0           51
  506                            Concerta     7.0           21
  520                     Methylphenidate     9.0          122
  594                     Methylphenidate    10.0           72
  664                            Daytrana     9.0           24
  697                            Daytrana    10.0           25
  780                             Vyvanse     5.0            6
  781                    Lisdexamfetamine    10.0           34
  834                             Ritalin     9.0           64
  835                              Kapvay    10.0           65
  845                           Strattera     5.0           15
  977                             Intuniv     3.0           19
  1018                           Adderall     6.0           34
  1038                        Atomoxetine     4.0            0
  1062                         Guanfacine     1.0            5
  1090    Amphetamine / dextroamphetamine    10.0            3
  1094                          Bupropion    10.0           30
  1109    Amphetamine / dextroamphetamine    10.0           51
  ...                                 ...     ...          ...
  159788                 Lisdexamfetamine     9.0           28
  159792  Amphetamine / dextroamphetamine     8.0           11
  159798                         Daytrana    10.0           15
  159802                 Lisdexamfetamine     9.0           38
  159849                          Ritalin     1.0            8
  159892                          Focalin     6.0           34
  159907                       Focalin XR     6.0           30
  160131                      Atomoxetine     6.0           35
  160146                          Mydayis     2.0           10
  160180                 Lisdexamfetamine     5.0            6
  160182                         Adderall     3.0           28
  160189                         Concerta     8.0           12
  160216                         Adderall     8.0           35
  160271                  Methylphenidate    10.0           32
  160275                          Focalin     1.0           16
  160309                        Strattera     8.0           25
  160355                      Adderall XR     3.0           14
  160439                      Atomoxetine    10.0           28
  160467                         Adderall     9.0           26
  160529  Amphetamine / dextroamphetamine     8.0           44
  160604                         Concerta     9.0           45
  160743                          Desoxyn     9.0          111
  160809                  Methamphetamine    10.0           96
  160839                           Cylert    10.0            4
  160848                         Adderall    10.0           59
  160977                 Lisdexamfetamine     8.0           36
  160998                         Adderall     8.0           50
  161084                  Methylphenidate     1.0           31
  161111                        Strattera     9.0           21
  161213  Amphetamine / dextroamphetamine     9.0            9
  
  [3383 rows x 3 columns],
  encoding: EncodingWithFacet({
    size: Size({
 

In [141]:
D = alt.Chart(adhd[feats_of_interest]).mark_point().encode(
  x = 'rating',
  y = 'count()',
  color = 'drugName',
  size = 'usefulCount',
  tooltip=['drugName', 'rating', 'count()']
).interactive()

D

Chart({
  data:                                drugName  rating  usefulCount
  1                            Guanfacine     8.0          192
  27                             Daytrana    10.0           11
  51                              Intuniv     1.0           23
  116                             Focalin     9.0           48
  211                             Vyvanse     9.0           11
  250                    Lisdexamfetamine    10.0           65
  268     Amphetamine / dextroamphetamine    10.0           15
  313                         Atomoxetine     9.0           23
  376     Amphetamine / dextroamphetamine    10.0            1
  382                            Concerta    10.0           43
  384     Amphetamine / dextroamphetamine     9.0           10
  385                           Strattera     3.0            8
  501                          Focalin XR     8.0           51
  506                            Concerta     7.0           21
  520                     Methylphenidate     9.0          122
  594                     Methylphenidate    10.0           72
  664                            Daytrana     9.0           24
  697                            Daytrana    10.0           25
  780                             Vyvanse     5.0            6
  781                    Lisdexamfetamine    10.0           34
  834                             Ritalin     9.0           64
  835                              Kapvay    10.0           65
  845                           Strattera     5.0           15
  977                             Intuniv     3.0           19
  1018                           Adderall     6.0           34
  1038                        Atomoxetine     4.0            0
  1062                         Guanfacine     1.0            5
  1090    Amphetamine / dextroamphetamine    10.0            3
  1094                          Bupropion    10.0           30
  1109    Amphetamine / dextroamphetamine    10.0           51
  ...                                 ...     ...          ...
  159788                 Lisdexamfetamine     9.0           28
  159792  Amphetamine / dextroamphetamine     8.0           11
  159798                         Daytrana    10.0           15
  159802                 Lisdexamfetamine     9.0           38
  159849                          Ritalin     1.0            8
  159892                          Focalin     6.0           34
  159907                       Focalin XR     6.0           30
  160131                      Atomoxetine     6.0           35
  160146                          Mydayis     2.0           10
  160180                 Lisdexamfetamine     5.0            6
  160182                         Adderall     3.0           28
  160189                         Concerta     8.0           12
  160216                         Adderall     8.0           35
  160271                  Methylphenidate    10.0           32
  160275                          Focalin     1.0           16
  160309                        Strattera     8.0           25
  160355                      Adderall XR     3.0           14
  160439                      Atomoxetine    10.0           28
  160467                         Adderall     9.0           26
  160529  Amphetamine / dextroamphetamine     8.0           44
  160604                         Concerta     9.0           45
  160743                          Desoxyn     9.0          111
  160809                  Methamphetamine    10.0           96
  160839                           Cylert    10.0            4
  160848                         Adderall    10.0           59
  160977                 Lisdexamfetamine     8.0           36
  160998                         Adderall     8.0           50
  161084                  Methylphenidate     1.0           31
  161111                        Strattera     9.0           21
  161213  Amphetamine / dextroamphetamine     9.0            9
  
  [3383 rows x 3 columns],
  encoding: EncodingWithFacet({
    color: Color({

In [97]:
adhd[feats_of_interest[1]]

1         ADHD
27        ADHD
51        ADHD
116       ADHD
211       ADHD
250       ADHD
268       ADHD
313       ADHD
376       ADHD
382       ADHD
384       ADHD
385       ADHD
501       ADHD
506       ADHD
520       ADHD
594       ADHD
664       ADHD
697       ADHD
780       ADHD
781       ADHD
834       ADHD
835       ADHD
845       ADHD
977       ADHD
1018      ADHD
1038      ADHD
1062      ADHD
1090      ADHD
1094      ADHD
1109      ADHD
          ... 
159788    ADHD
159792    ADHD
159798    ADHD
159802    ADHD
159849    ADHD
159892    ADHD
159907    ADHD
160131    ADHD
160146    ADHD
160180    ADHD
160182    ADHD
160189    ADHD
160216    ADHD
160271    ADHD
160275    ADHD
160309    ADHD
160355    ADHD
160439    ADHD
160467    ADHD
160529    ADHD
160604    ADHD
160743    ADHD
160809    ADHD
160839    ADHD
160848    ADHD
160977    ADHD
160998    ADHD
161084    ADHD
161111    ADHD
161213    ADHD
Name: condition, Length: 3383, dtype: object

In [0]:

# # Plot the data points on an interactive axis
# points = alt.Chart(df).mark_circle(color='black').encode(
#     x=alt.X('x', axis=alt.Axis(title='x')),
#     y=alt.Y('y', axis=alt.Axis(title='y')),
# ).interactive()

# # Plot the best fit polynomials
# polynomial_fit = alt.Chart(poly_data).mark_line().encode(
#     x='xfit',
#     y='yfit',
#     color='degree'
# )

# points + polynomial_fit


In [0]:

def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
  return (mean, mean - interval, mean + interval)
